## SRJ

## NH3 - Ham verification

In [ ]:
'N 0 0 0; H 0 0 1.40; H 0 0.60 -0.336000; H 0.60 0.0 -0.336000'

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    core, active = qml.qchem.active_space(electrons, orbitals, active_electrons=active_electrons, active_orbitals=active_orbitals) # Spatial orbitals
    print('core orbitals:', core)
    print('active orbitals:', active)
    #print(H)
    return core, active

In [2]:
symbols  = [ 'N', 'H', 'H', 'H']
print('H2O-0.6A-GS+ES-DBS')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 1.40*r_bohr], [0, 0.60*r_bohr, -0.336*r_bohr], [0.60, 0, -0.336*r_bohr]])

print(coordinates)

electrons = 10  # 7 from N and 3 from H
orbitals = 16 #Spin orbitals  
charge = 0

active_electrons = 6 #No of electrons
active_orbitals = 6 #Spatial orbitals

core, active = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=1)

H2O-0.6A-GS+ES-DBS
[[ 0.          0.          0.        ]
 [ 0.          0.          2.64561638]
 [ 0.          1.13383559 -0.63494793]
 [ 0.6         0.         -0.63494793]]
Using active space, check if you change the H accordingly
core orbitals: [0, 1]
active orbitals: [2, 3, 4, 5, 6, 7]


In [3]:
core
active

[2, 3, 4, 5, 6, 7]

In [4]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged) 
mol_pyscf = gto.M(
    atom = 'N 0 0 0; H 0 0 1.40; H 0 0.60 -0.336000; H 0.60 0.0 -0.336000',
    basis = 'sto-3g',
    spin = 0)
rhf = scf.RHF(mol_pyscf)
rhf.kernel()

import pennylane as qml 
from pennylane import qchem


one_ao = mol_pyscf.intor_symmetric("int1e_kin") + mol_pyscf.intor_symmetric("int1e_nuc")
two_ao = mol_pyscf.intor("int2e_sph")


one_mo = np.einsum("pi,pq,qj->ij", rhf.mo_coeff, one_ao, rhf.mo_coeff, optimize=True)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

core_constant = np.array([rhf.energy_nuc()])
print('The value of core:', core_constant)

# convert the two-electron integral tensor to the physicists’ notation
two_mo = np.swapaxes(two_mo, 1, 3)

#Changed core and active orbitals


total_orb = len(core) + len(active)
print('Total orbitals are',total_orb)

# define the active space and recompute the integrals
def pyscf_integral(core, active, one_mo, two_mo, core_constant):
    if core and active:
        for i in core:
            core_constant = core_constant + 2 * one_mo[i][i]
            for j in core:
                core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]

        for p in active:
            for q in active:
                for i in core:
                    print(f'The value of p and q', {p}, {q})
                    one_mo[p, q] = one_mo[p, q] + (2 * two_mo[i][p][q][i] - two_mo[i][p][i][q])

        one_mo = one_mo[qml.math.ix_(active, active)]
        two_mo = two_mo[qml.math.ix_(active, active, active, active)]

    return core_constant, one_mo, two_mo

core_constant, one_mo, two_mo = pyscf_integral(core, active, one_mo,two_mo, core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)
print(H)

converged SCF energy = -54.783787420061
The value of core: [14.61897422]
Total orbitals are 8
The value of p and q {2} {2}
The value of p and q {2} {2}
The value of p and q {2} {3}
The value of p and q {2} {3}
The value of p and q {2} {4}
The value of p and q {2} {4}
The value of p and q {2} {5}
The value of p and q {2} {5}
The value of p and q {2} {6}
The value of p and q {2} {6}
The value of p and q {2} {7}
The value of p and q {2} {7}
The value of p and q {3} {2}
The value of p and q {3} {2}
The value of p and q {3} {3}
The value of p and q {3} {3}
The value of p and q {3} {4}
The value of p and q {3} {4}
The value of p and q {3} {5}
The value of p and q {3} {5}
The value of p and q {3} {6}
The value of p and q {3} {6}
The value of p and q {3} {7}
The value of p and q {3} {7}
The value of p and q {4} {2}
The value of p and q {4} {2}
The value of p and q {4} {3}
The value of p and q {4} {3}
The value of p and q {4} {4}
The value of p and q {4} {4}
The value of p and q {4} {5}
The val

# Verification of HF energy

In [5]:
qubits = 12
active_electrons = 6
hf_state = qchem.hf_state(active_electrons, qubits)
dev = qml.device("lightning.qubit", wires = qubits)
@qml.qnode(dev)
def e():
    qml.BasisState(hf_state, wires=range(qubits))
    return qml.expval(H)

fe = e()
print('fe', fe)

fe -54.7837874200609


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


In [6]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
#warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(active_electrons, H, adapt_it):
    print('Using active space, check if you change the H accordingly')
    qubits = 2*active_orbitals
    active_electrons = active_electrons
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #hf_state = np.array([1,1,1,0,0,0,1,1,1,0,0,0])
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad






#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0




gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact( active_electrons, H, adapt_it=25) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


Using active space, check if you change the H accordingly
HF state is -54.7837874200609
Total excitations are 117
The adapt iteration now is 0


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.33410282320961726 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [0.31918171]
Final cost: -54.81067614505366
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(0.32)─┤  State
3: ──X─├FermionicDoubleExcitation(0.32)─┤  State
4: ──X─│────────────────────────────────┤  State
5: ──X─│────────────────────────────────┤  State
6: ────├FermionicDoubleExcitation(0.32)─┤  State
7: ────╰FermionicDoubleExcitation(0.32)─┤  State
The adapt iteration now is 1


...

The highest operator value is 0.029915797402964923 for operator a⁺(0) a⁺(3) a(8) a(11)
Highest gradient excitation is [0, 3, 8, 11]
Final updated parameters: [ 0.31048922  0.09931488  0.1324854  -0.13278352  0.09947304  0.03590064
  0.02868416  0.02842219  0.02842665  0.02770641  0.03842257  0.03842444
  0.03312091 -0.03312645  0.02248292  0.01726715 -0.01970331  0.01971472
  0.01606958  0.01605504 -0.02550324 -0.02550067 -0.01237691 -0.01237935]
Final cost: -54.834354313326166
 0: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.10) ···
 1: ──X──────────────────────────────────├FermionicDoubleExcitation(0.10) ···
 2: ──X─╭FermionicDoubleExcitation(0.31)─│─────────────────────────────── ···
 3: ──X─├FermionicDoubleExcitation(0.31)─│─────────────────────────────── ···
 4: ──X─│────────────────────────────────│─────────────────────────────── ···
 5: ──X─│────────────────────────────────│─────────────────────────────── ···
 6: ────├FermionicDoubleExcitation(0.31)─│───────

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.028518341511273734 for operator a⁺(1) a⁺(4) a(8) a(11)
Highest gradient excitation is [1, 4, 8, 11]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/s.poyyapakkam

Final updated parameters: [ 0.31050464  0.09923353  0.13249328 -0.1327605   0.09947158  0.03581039
  0.02868603  0.02846932  0.02842698  0.02770685  0.03842791  0.0384548
  0.03313776 -0.03310987  0.02248312  0.01726047 -0.01970285  0.01978741
  0.0160593   0.01605048 -0.02549612 -0.02549275 -0.01236349 -0.01233539
 -0.01250575]
Final cost: -54.83444349038657
 0: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.10) ···
 1: ──X──────────────────────────────────├FermionicDoubleExcitation(0.10) ···
 2: ──X─╭FermionicDoubleExcitation(0.31)─│─────────────────────────────── ···
 3: ──X─├FermionicDoubleExcitation(0.31)─│─────────────────────────────── ···
 4: ──X─│────────────────────────────────│─────────────────────────────── ···
 5: ──X─│────────────────────────────────│─────────────────────────────── ···
 6: ────├FermionicDoubleExcitation(0.31)─│─────────────────────────────── ···
 7: ────╰FermionicDoubleExcitation(0.31)─│─────────────────────────────── ···
 8: ──────────